In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import pandas as pd

In [2]:
def get_info(film_tag):
    name_tag = film_tag.find('h3')
    name = name_tag.find('a').get_text()
    year = name_tag.find('span', class_ = 'lister-item-year text-muted unbold').get_text()
    year = int(year[-5 : -1])
    
    muted_text = film_tag.find_all( 'p', class_ = 'text-muted')
    overview = muted_text[1].get_text().strip()
    
    cer = film_tag.find('span', class_ = 'certificate')
    if cer:
        certificate = cer.get_text()
    else:
        certificate = ''
    
    runtime_tag = film_tag.find('span', class_ = 'runtime')
    if runtime_tag:
        runtime = float(runtime_tag.get_text().split(' ')[0])
    else:
        runtime = np.nan
        
    genre_tag = film_tag.find('span', class_ = 'genre')
    if genre_tag:
        genre = genre_tag.get_text().strip().split(', ')
        genre = ', '.join(genre)
    else: 
        genre = ''
    
    imdb_rate_tag = film_tag.find('strong')
    if imdb_rate_tag:
        imdb_rate = float(imdb_rate_tag.get_text())
    else:
        imdb_rate = np.nan
    metascore_tag = film_tag.find('span', class_ = 'metascore favorable')
    if metascore_tag :
        metascore = float(metascore_tag.get_text().strip())
    else:
        metascore =np.nan
    star_tag = film_tag.find('p', class_ = "").get_text().strip()
    star_tag = star_tag.split('|')
    if len(star_tag) > 1:
        director_tag =  star_tag[0].split(':\n')
        star_tag = star_tag[1].split(':\n')
        director = director_tag[1].strip().split(', \n')
        director = ', '.join(director)
        stars = star_tag[1].strip().split(', \n')
        stars = ', '.join(stars)
    else:
        temp = star_tag[0].strip().split(':\n')
        if temp[0] == 'Director':
            director = temp[1]
            stars = ''
        else:
            stars = temp[1]
            director = ''
        
    vote_tag = film_tag.find('p', class_ = 'sort-num_votes-visible').get_text()
    vote_tag = vote_tag.split('\n|')
    nvote = vote_tag[0].split(':\n')[1].replace(',', '')
    nvote = float(nvote)
    if len(vote_tag) > 1:
        gross = float(vote_tag[1].strip().split(':\n')[1].strip('$M'))
    else:
        gross = np.nan
    
    result = [name, year, overview, certificate, runtime, genre, imdb_rate, metascore, director, stars, nvote, gross]
    return result

In [3]:
film_list = []
# search feature film có nhiều lượt vote từ 2010 - 2021, 
url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2010-01-01,2021-12-31&sort=num_votes,desc&count=250"
while len(film_list) < 1000:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    scraped_list = soup.find_all( 'div', class_ = 'lister-item-content')
    for film in scraped_list:
        film_list.append(get_info(film))
    next_tag = soup.find('a', class_ = 'lister-page-next next-page')
    url = 'https://www.imdb.com/' + next_tag.get('href')
key = ['name', 'year', 'overview', 'certificate', 'runtime', 'genre', 'imdb_rate', 'metascore', 'director', 'stars', 'nvote', 'gross']
df = pd.DataFrame(film_list,columns = key)

In [4]:
df

,name,year,overview,certificate,runtime,genre,imdb_rate,metascore,director,stars,nvote,gross
0,Kẻ Đánh Cắp Giấc Mơ,2010,A thief who steals corporate secrets through t...,PG-13,148.0,"Action, Adventure, Sci-Fi",8.8,74.0,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",2343355.0,292.58
1,Hố Đen Tử Thần,2014,A team of explorers travel through a wormhole ...,C13,169.0,"Adventure, Drama, Sci-Fi",8.6,74.0,Christopher Nolan,"Matthew McConaughey, Anne Hathaway, Jessica Ch...",1819569.0,188.02
2,Kỵ Sĩ Bóng Đêm Trỗi Dậy,2012,Eight years after the Joker's reign of anarchy...,PG-13,164.0,"Action, Drama",8.4,78.0,Christopher Nolan,"Christian Bale, Tom Hardy, Anne Hathaway, Gary...",1700913.0,448.14
3,Hành Trình Django,2012,"With the help of a German bounty-hunter, a fre...",R,165.0,"Drama, Western",8.4,81.0,Quentin Tarantino,"Jamie Foxx, Christoph Waltz, Leonardo DiCaprio...",1548513.0,162.81
4,Sói Già Phố Wall,2013,"Based on the true story of Jordan Belfort, fro...",R,180.0,"Biography, Comedy, Crime",8.2,75.0,Martin Scorsese,"Leonardo DiCaprio, Jonah Hill, Margot Robbie, ...",1403899.0,116.90
...,...,...,...,...,...,...,...,...,...,...,...,...
995,To Rome with Love,2012,The lives of some visitors and residents of Ro...,R,112.0,"Comedy, Music, Romance",6.3,NaN,Woody Allen,"Woody Allen, Penélope Cruz, Jesse Eisenberg, E...",88530.0,16.69
996,Niệm Khúc Cuối,2010,A rebellious girl is sent to a Southern beach ...,PG,108.0,"Drama, Music, Romance",6.0,NaN,Julie Anne Robinson,"Miley Cyrus, Liam Hemsworth, Greg Kinnear, Bob...",88482.0,62.95
997,Cậu Ấm Quái Chiêu,2011,A comedy centered on an idealist who barges in...,R,90.0,"Comedy, Drama",6.4,NaN,Jesse Peretz,"Paul Rudd, Elizabeth Banks, Zooey Deschanel, N...",88479.0,24.81
998,Rắc Rối Tuổi Teen,2013,A 20-something supervising staff member of a r...,R,96.0,Drama,7.9,82.0,Destin Daniel Cretton,"Brie Larson, Frantz Turner, John Gallagher Jr....",88413.0,1.01


In [5]:
df.to_csv('data.csv', index = False, sep='\t', encoding='utf-8')